In [0]:
from pyspark.sql.functions import current_timestamp
from pyspark.sql import functions as F


In [0]:
landing_path = "/Volumes/mlb/00_landing/data_sources/player_name_lookup/people-*.csv"
bronze_path = "mlb.01_bronze.player_name"
task_name = "00_Ingest_PlayerName_Lookup" 


In [0]:
continue_flag = dbutils.jobs.taskValues.get(
    taskKey=task_name, 
    key="continue_downstream", 
    default="no"
)

if continue_flag == "no":
    dbutils.notebook.exit("Landing failed or skipped. Stopping Bronze process.")

In [0]:
bronze_df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "false") \
    .load(landing_path)

In [0]:
# bronze_df.display()

In [0]:
bronze_df_with_meta = bronze_df \
    .withColumn("ingestion_timestamp", F.current_timestamp()) \
    .select("*", F.col("_metadata.file_path").alias("source_file"))

In [0]:
bronze_df_with_meta.write.format("delta") \
    .mode("append") \
    .saveAsTable(bronze_path)